In [2]:
import tensorflow as tf
import numpy as np
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Dropout, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
!pip install pydub
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
state="yes no"
state=state.split()
directory = "/content/gdrive/My Drive/anxiety/train/"
for s in state:
  filenames = os.listdir(os.path.join(directory,f"{s}"))
  random.shuffle(filenames)
  test_files = filenames[0:100]

  for f in test_files:

    shutil.move(directory + f"{s}"+ "/" + f,"/content/gdrive/My Drive/anxiety/test/" + f"{s}")

In [3]:
train_dir = "/content/gdrive/My Drive/anxiety/train/"
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(288,432),color_mode="rgba",class_mode='categorical',batch_size=128)

validation_dir = "/content/gdrive/My Drive/anxiety/test/"
vali_datagen = ImageDataGenerator(rescale=1./255)
vali_generator = vali_datagen.flow_from_directory(validation_dir,target_size=(288,432),color_mode='rgba',class_mode='categorical',batch_size=128)

Found 600 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [4]:
def AModel(input_shape = (288,432,4),classes=2):
  
  X_input = Input(input_shape)

  X = Conv2D(8,kernel_size=(3,3),strides=(1,1))(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(16,kernel_size=(3,3),strides = (1,1))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(32,kernel_size=(3,3),strides = (1,1))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64,kernel_size=(3,3),strides=(1,1))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  X = Conv2D(128,kernel_size=(3,3),strides=(1,1))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  
  X = Flatten()(X)
  
  X = Dropout(rate=0.3)(X)
  X=  Dense(128,activation="relu")(X)

  X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)

  model = Model(inputs=X_input,outputs=X,name='AModel')

  return model

In [5]:
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
  
model = AModel(input_shape=(288,432,4),classes=2)
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy',metrics=['accuracy',get_f1]) 

model.fit_generator(train_generator,epochs=10,validation_data=vali_generator)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  from ipykernel import kernelapp as app


Epoch 1/10
5/5 [==============================] - 80s 16s/step - loss: 2.8109 - accuracy: 0.5417 - get_f1: 0.5455 - val_loss: 0.6992 - val_accuracy: 0.3333 - val_get_f1: 0.3002
Epoch 2/10
5/5 [==============================] - 74s 15s/step - loss: 0.9658 - accuracy: 0.5117 - get_f1: 0.5180 - val_loss: 0.6951 - val_accuracy: 0.3333 - val_get_f1: 0.3449
Epoch 3/10
5/5 [==============================] - 76s 16s/step - loss: 0.7078 - accuracy: 0.5050 - get_f1: 0.5018 - val_loss: 0.6902 - val_accuracy: 0.6667 - val_get_f1: 0.6650
Epoch 4/10
5/5 [==============================] - 73s 16s/step - loss: 0.6729 - accuracy: 0.5000 - get_f1: 0.4986 - val_loss: 0.6924 - val_accuracy: 0.6667 - val_get_f1: 0.6899
Epoch 5/10
5/5 [==============================] - 82s 17s/step - loss: 0.6426 - accuracy: 0.5000 - get_f1: 0.5057 - val_loss: 0.6931 - val_accuracy: 0.6667 - val_get_f1: 0.6948
Epoch 6/10
5/5 [==============================] - 73s 15s/step - loss: 0.6399 - accuracy: 0.5433 - get_f1: 0.5463 -